## Задание 5.1


Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from pymorphy2 import MorphAnalyzer

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", sep="\t")
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [3]:
df.sentiment.value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [4]:
df['tokens'] = df['review'].transform(word_tokenize)
df.head()

,review,sentiment,tokens
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, (, горловин..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[Товар, отдали, другому, человеку, ,, я, не, п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[Ужасная, синтетика, !, Тонкая, ,, ничего, общ..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, пришел, ,, продавец, продлил, защи..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[Кофточка, голая, синтетика, ,, носить, не, во..."


Приведение к нижнему регистру

In [5]:
df['tokens'] = df['tokens'].apply(lambda x: list(map(lambda y: y.lower(), x)))
df['tokens'].head()

0    [качество, плохое, пошив, ужасный, (, горловин...
1    [товар, отдали, другому, человеку, ,, я, не, п...
2    [ужасная, синтетика, !, тонкая, ,, ничего, общ...
3    [товар, не, пришел, ,, продавец, продлил, защи...
4    [кофточка, голая, синтетика, ,, носить, не, во...
Name: tokens, dtype: object

Удаление стоп-слов

In [6]:
def remove_sth(series, for_remove):
    return series.apply(lambda x: list(filter(lambda y: y not in for_remove, x)))

In [7]:
df['tokens'] = remove_sth(df['tokens'], string.punctuation)
df['tokens'].head()

0    [качество, плохое, пошив, ужасный, горловина, ...
1    [товар, отдали, другому, человеку, я, не, полу...
2    [ужасная, синтетика, тонкая, ничего, общего, с...
3    [товар, не, пришел, продавец, продлил, защиту,...
4    [кофточка, голая, синтетика, носить, не, возмо...
Name: tokens, dtype: object

Удаление стоп-слов

In [8]:
noise = stopwords.words('russian')
df['tokens']=remove_sth(df['tokens'], noise)

In [9]:
morph_analyzer=MorphAnalyzer()
df['tokens']=df['tokens'].apply(lambda x: list(map(lambda y: morph_analyzer.parse(y)[0].normal_form, x)))
df['tokens'].head()

0    [качество, плохой, пошив, ужасный, горловина, ...
1    [товар, отдать, другой, человек, получить, пос...
2    [ужасный, синтетик, тонкий, общий, представить...
3    [товар, прийти, продавец, продлить, защита, мо...
4        [кофточка, голый, синтетик, носить, возможно]
Name: tokens, dtype: object

### Векторизация (с настройкой гиперпараметров) и построение модели

Необходимо использовать следующие веторизаторы:
1. Мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. TF-IDF (диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. Символьные n-граммы (диапазон для n подбирайте самостоятельно)

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [11]:
df['strings'] = df['tokens'].apply(lambda x: ' '.join(x))
df['strings'].head()

0    качество плохой пошив ужасный горловина напере...
1    товар отдать другой человек получить посылка л...
2    ужасный синтетик тонкий общий представить карт...
3    товар прийти продавец продлить защита мой согл...
4              кофточка голый синтетик носить возможно
Name: strings, dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.strings, df.sentiment, train_size=0.66, random_state=42)

Count vectorizer

In [13]:
pipeline = Pipeline([("vectorizer", CountVectorizer()),("classificator", MultinomialNB())])
cv = GridSearchCV(pipeline, param_grid={'vectorizer__ngram_range': [(1, i) for i in range(2, 10)]})
cv.fit(X_train, y_train)
cv.best_estimator_

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('classificator', MultinomialNB())])

In [14]:
print(classification_report(y_test, cv.predict(X_test)))

              precision    recall  f1-score   support

    neautral       0.61      0.63      0.62     10328
    negative       0.71      0.66      0.68     10126
    positive       0.82      0.85      0.84     10146

    accuracy                           0.71     30600
   macro avg       0.71      0.71      0.71     30600
weighted avg       0.71      0.71      0.71     30600



TF-IDF

In [15]:
pipeline1=Pipeline([("vectorizer", TfidfVectorizer()),("classificator", MultinomialNB())])
cv1 = GridSearchCV(pipeline1, param_grid={
    "vectorizer__max_df": (0.5, 0.75, 0.95),
    "vectorizer__min_df": (0, 0.01, 0.1, 0.25),
    'vectorizer__max_features': (None, 1000, 5000, 10000),
    "vectorizer__ngram_range": ((1, 2), (1, 3))
})
cv1.fit(X_train, y_train)
cv1.best_estimator_

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.5, min_df=0, ngram_range=(1, 2))),
                ('classificator', MultinomialNB())])

In [16]:
print(classification_report(y_test, cv1.predict(X_test)))

              precision    recall  f1-score   support

    neautral       0.61      0.63      0.62     10328
    negative       0.71      0.66      0.68     10126
    positive       0.82      0.86      0.84     10146

    accuracy                           0.71     30600
   macro avg       0.71      0.71      0.71     30600
weighted avg       0.71      0.71      0.71     30600



Char vectorizer

In [17]:
pipeline2=Pipeline([("vectorizer", CountVectorizer(analyzer='char')),("classificator", MultinomialNB())])
cv2 = GridSearchCV(pipeline2, param_grid={"vectorizer__ngram_range": [(3,4), (5,7), (9,15),(16,17)]})
cv2.fit(X_train, y_train)
cv2.best_estimator_

Pipeline(steps=[('vectorizer',
                 CountVectorizer(analyzer='char', ngram_range=(5, 7))),
                ('classificator', MultinomialNB())])

In [18]:
print(classification_report(y_test, cv2.predict(X_test)))

              precision    recall  f1-score   support

    neautral       0.59      0.66      0.62     10328
    negative       0.71      0.63      0.66     10126
    positive       0.83      0.83      0.83     10146

    accuracy                           0.71     30600
   macro avg       0.71      0.71      0.71     30600
weighted avg       0.71      0.71      0.71     30600



In [19]:
def get_scores(models, X_test, y_test, row_names):
    values=[]
    for model in models:
        precision = precision_score(y_test, model.predict(X_test), average='weighted')
        recall = recall_score(y_test, model.predict(X_test), average='weighted')
        f1 = f1_score(y_test, model.predict(X_test), average='weighted')
        accuracy = accuracy_score(y_test, model.predict(X_test))
        values.append([precision,recall,f1,accuracy])
    return pd.DataFrame(values, columns=['precision', 'recall', 'f1', 'accuracy'], index=row_names)

In [20]:
print(get_scores([cv, cv1, cv2], X_test, y_test, ['Count vectorizer', 'TF-IDF', 'Char vectorizer']))

                  precision    recall        f1  accuracy
Count vectorizer   0.712426  0.712451  0.711975  0.712451
TF-IDF             0.713018  0.714020  0.712972  0.714020
Char vectorizer    0.709840  0.705033  0.706006  0.705033


Как мы видим, все три модели показали примерно одинаковый результат c точностью порядка 70-71%.

## Задание 5.2 Регулярные выражения

In [21]:
import re

### findall

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [33]:
random_words = 'neoclassical resolute hairdressing salon beauty parlour wrapping paper gardenia multicereal porridge'
print(re.findall(r'\b[a-zA-Z]{2}', random_words))

['ne', 're', 'ha', 'sa', 'be', 'pa', 'wr', 'pa', 'ga', 'mu', 'po']


### split

**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [34]:
random_sentence = 'neoclassical resolute. hairdressing salon. beauty. parlour wrapping. paper gardenia. multicereal porridge'
print(re.split(r'\. ', random_sentence, maxsplit=2))

['neoclassical resolute', 'hairdressing salon', 'beauty. parlour wrapping. paper gardenia. multicereal porridge']


### sub

**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [28]:
digit_string = 'Some phone number: 8-921-887-39-20'
print(re.sub('\d', 'DIG', digit_string))

Some phone number: DIG-DIGDIGDIG-DIGDIGDIG-DIGDIG-DIGDIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [35]:
url = 'Visit my github: https://github.com/imashevchenko'
regex = r'(https://)?[\w\.-]+\.\w{2,3}(/[\w-]*(\.[\w-]+)?)*'

print(re.sub(regex, 'URL', url))

Visit my github: URL


### compile

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [36]:
random_sentence = 'neoclassical resolute hairdressing salon beauty parlour wrapping paper gardenia multicereal porridge'
regex_obj = re.compile(r'\w{4,}')
regex_obj.findall(random_sentence)

['neoclassical',
 'resolute',
 'hairdressing',
 'salon',
 'beauty',
 'parlour',
 'wrapping',
 'paper',
 'gardenia',
 'multicereal',
 'porridge']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

In [40]:
email_string = 'shevchenkoav2002@gmail.com, vitars@gmail.com, smth.smth@yandex.ru'
regex_obj_2 = re.compile(r'@\w+\.\w+')
regex_obj_2.findall(email_string)

['@gmail.com', '@gmail.com', '@yandex.ru']